Importing ***PySpark*** and other libraries.

In [4]:
!pip install pyspark
import pyspark
from pyspark.sql import SparkSession

Creating a Spark session.

In [5]:
spark = SparkSession.builder.appName("StatisticalAnalysisApp").getOrCreate()
spark

Reading a cleaned and preprocessed dataset from Google Drive titled "***finalcleaned3 (1)***".

In [6]:
ds = spark.read.csv("/content/drive/MyDrive/Cloud Technologies/Notebooks/finalcleaned3 (1).csv", header=True, inferSchema=True)


Showing the Dataset. Showing first 20 rows.

In [7]:
ds.show()

+---------------+-----------+-------+--------+----------------------------+-----------------------------+-------------------------------------------------------+-------------------------------------------------------+-------------+-----------------------+-----------+--------------------+------------------+-----------------------+
|header.location|map.country|map.lat| map.lng|salary.salaries.val.jobTitle|salary.salaries.val.payPeriod|salary.salaries.val.salaryPercentileMap.payPercentile10|salary.salaries.val.salaryPercentileMap.payPercentile90|    Continent|adjusted_pay_percentile|    Country|Cost of Living Index| percentile10by100|Purchasing Power Parity|
+---------------+-----------+-------+--------+----------------------------+-----------------------------+-------------------------------------------------------+-------------------------------------------------------+-------------+-----------------------+-----------+--------------------+------------------+-----------------------+
|   

Now Showing top 30 rows.

In [8]:
ds.show(30)

+---------------+--------------+-------+--------+----------------------------+-----------------------------+-------------------------------------------------------+-------------------------------------------------------+-------------+-----------------------+--------------+--------------------+------------------+-----------------------+
|header.location|   map.country|map.lat| map.lng|salary.salaries.val.jobTitle|salary.salaries.val.payPeriod|salary.salaries.val.salaryPercentileMap.payPercentile10|salary.salaries.val.salaryPercentileMap.payPercentile90|    Continent|adjusted_pay_percentile|       Country|Cost of Living Index| percentile10by100|Purchasing Power Parity|
+---------------+--------------+-------+--------+----------------------------+-----------------------------+-------------------------------------------------------+-------------------------------------------------------+-------------+-----------------------+--------------+--------------------+------------------+-----------

Creating a partition of 4 for our dataset and naming it ds_par.

In [9]:
ds_par = ds.repartition(4)

Showing number of Partitions.

In [10]:
num = ds_par.rdd.getNumPartitions()
print(f"The number of partitions : {num}")

The number of partitions : 4


Name of columns.

In [11]:
ds_par.columns

['header.location',
 'map.country',
 'map.lat',
 'map.lng',
 'salary.salaries.val.jobTitle',
 'salary.salaries.val.payPeriod',
 'salary.salaries.val.salaryPercentileMap.payPercentile10',
 'salary.salaries.val.salaryPercentileMap.payPercentile90',
 'Continent',
 'adjusted_pay_percentile',
 'Country',
 'Cost of Living Index',
 'percentile10by100',
 'Purchasing Power Parity']

Renaming some columns for clarity.

In [12]:

ds_par = ds_par.withColumnRenamed("salary.salaries.val.jobTitle","Job Title")
ds_par = ds_par.withColumnRenamed("salary.salaries.val.salaryPercentileMap.payPercentile10","Salary-Percentile-10")
ds_par = ds_par.withColumnRenamed("salary.salaries.val.salaryPercentileMap.payPercentile90","Salary-Percentile-90")
ds_par = ds_par.withColumnRenamed("salary.salaries.val.payPeriod","Pay Period")

In [13]:
ds_par.show(10)

+---------------+--------------+--------+---------+--------------------+----------+--------------------+--------------------+-------------+-----------------------+--------------+--------------------+------------------+-----------------------+
|header.location|   map.country| map.lat|  map.lng|           Job Title|Pay Period|Salary-Percentile-10|Salary-Percentile-90|    Continent|adjusted_pay_percentile|       Country|Cost of Living Index| percentile10by100|Purchasing Power Parity|
+---------------+--------------+--------+---------+--------------------+----------+--------------------+--------------------+-------------+-----------------------+--------------+--------------------+------------------+-----------------------+
|           pune|         India| 18.5042|  73.8531|Regional Sales Di...|    ANNUAL|           111909.88|           130831.55|         Asia|              242741.43|         India|                21.2|       1.1190988E7|      527876.7924528302|
|        midrand|  South Afr

Data Types of Dataset.

In [14]:
ds_par.dtypes

[('header.location', 'string'),
 ('map.country', 'string'),
 ('map.lat', 'double'),
 ('map.lng', 'double'),
 ('Job Title', 'string'),
 ('Pay Period', 'string'),
 ('Salary-Percentile-10', 'double'),
 ('Salary-Percentile-90', 'double'),
 ('Continent', 'string'),
 ('adjusted_pay_percentile', 'double'),
 ('Country', 'string'),
 ('Cost of Living Index', 'double'),
 ('percentile10by100', 'double'),
 ('Purchasing Power Parity', 'double')]

In [15]:
from pyspark.sql.functions import col, when

Making Annual Salary columns titled ***Annual-Salary-Percentile-10*** and ***Annual-Salary-Percentile-90*** from ***Salary-Percentile-10*** and ***Salary-Percentile-90***.<br>
For monthly salary 12 has been multiplied and for hourly, we assumed 52 weeks and 40 hours per week working period and multiplied accordingly.

In [16]:
ds_par_s = ds_par.withColumn(
    "Annual-Salary-Percentile-10",
    when(col("Pay Period") == "MONTHLY", col("Salary-Percentile-10") * 12)
    .when(col("Pay Period") == "HOURLY", col("Salary-Percentile-10") * 52 * 40)
    .otherwise(col("Salary-Percentile-10")) )

ds_par_s = ds_par_s.withColumn(
    "Annual-Salary-Percentile-90",
    when(col("Pay Period") == "MONTHLY", col("Salary-Percentile-90") * 12)
    .when(col("Pay Period") == "HOURLY", col("Salary-Percentile-90") * 52 * 40)
    .otherwise(col("Salary-Percentile-90")) )


Transformed new columns.

In [17]:
ds_par_s.select("Annual-Salary-Percentile-10", "Annual-Salary-Percentile-90", "Pay Period").show()

+---------------------------+---------------------------+----------+
|Annual-Salary-Percentile-10|Annual-Salary-Percentile-90|Pay Period|
+---------------------------+---------------------------+----------+
|                   83240.03|                   90787.02|    ANNUAL|
|                   28093.74|                   30068.25|    ANNUAL|
|                   28089.87|                   30465.82|    ANNUAL|
|                   14992.66|                   17373.25|    ANNUAL|
|                    41725.4|                    45138.7|    ANNUAL|
|                  104553.18|                  114050.72|    ANNUAL|
|                  104553.18|                  114050.72|    ANNUAL|
|                   62146.75|                   67218.61|    ANNUAL|
|                    69099.5|                   75552.62|    ANNUAL|
|                   15913.16|                   18764.68|    ANNUAL|
|                   39376.95|                   61851.01|    ANNUAL|
|                   78050.63|     

Creating a new column titled ***Mean-Annual-Salary*** from ***Annual-Salary-Percentile-10*** and ***Annual-Salary-Percentile-90*** by weighting them.

In [18]:
ds_par_ms = ds_par_s.withColumn(
    "Mean-Annual-Salary",
    (col("Annual-Salary-Percentile-10") * 0.1 + col("Annual-Salary-Percentile-90") * 0.9) )

In [19]:
ds_par_ms.show()

+-------------------+--------------+--------+---------+--------------------+----------+--------------------+--------------------+-------------+-----------------------+--------------+--------------------+------------------+-----------------------+---------------------------+---------------------------+------------------+
|    header.location|   map.country| map.lat|  map.lng|           Job Title|Pay Period|Salary-Percentile-10|Salary-Percentile-90|    Continent|adjusted_pay_percentile|       Country|Cost of Living Index| percentile10by100|Purchasing Power Parity|Annual-Salary-Percentile-10|Annual-Salary-Percentile-90|Mean-Annual-Salary|
+-------------------+--------------+--------+---------+--------------------+----------+--------------------+--------------------+-------------+-----------------------+--------------+--------------------+------------------+-----------------------+---------------------------+---------------------------+------------------+
|               pune|         Indi

Saving the dataset with file name **Dataset with Mean Salary** to Google Drive

In [22]:
ds_par_ms.write.csv("/content/drive/MyDrive/Cloud Technologies/Notebooks/Transformed Datasets/Dataset with Mean Salary", mode="overwrite", header=True)


In [23]:
ds_par_ms.columns

['header.location',
 'map.country',
 'map.lat',
 'map.lng',
 'Job Title',
 'Pay Period',
 'Salary-Percentile-10',
 'Salary-Percentile-90',
 'Continent',
 'adjusted_pay_percentile',
 'Country',
 'Cost of Living Index',
 'percentile10by100',
 'Purchasing Power Parity',
 'Annual-Salary-Percentile-10',
 'Annual-Salary-Percentile-90',
 'Mean-Annual-Salary']

Choosing necessary columns and makin a new dataset titled, ***ds_par_ms_s***.

In [24]:
ds_par_ms_s = ds_par_ms.select("Job Title","Cost of Living Index", "Mean-Annual-Salary", "Country", "Continent")

In [25]:
ds_par_ms_s.show()

+--------------------+--------------------+------------------+-------------+-------------+
|           Job Title|Cost of Living Index|Mean-Annual-Salary|      Country|    Continent|
+--------------------+--------------------+------------------+-------------+-------------+
|               C O O|                56.2|232716.11200000002|        Italy|       Europe|
|Senior Software E...|                47.3|         77999.141|        Spain|       Europe|
|Senior Software E...|                47.3|         77999.141|        Spain|       Europe|
|Senior Software E...|                47.3|         77999.141|        Spain|       Europe|
|          Consultant|                31.7|61709.539000000004|        China|         Asia|
|          Consultant|                31.7|61709.539000000004|        China|         Asia|
|          Consultant|                31.7|61709.539000000004|        China|         Asia|
|          Consultant|                31.7|61709.539000000004|        China|         Asia|

Saving the dataset with file name **Dataset with Chosen Columns** to Google Drive

In [28]:
ds_par_ms_s.write.csv("/content/drive/MyDrive/Cloud Technologies/Notebooks/Transformed Datasets/Dataset with Chosen Columns",mode="overwrite", header=True)


The dataset ***ds_par_ms_s*** is sorted into dataset ***ds_par_ms_sli*** based on **Cost of Living Index**

In [29]:
ds_par_ms_sli = ds_par_ms_s.sort("Cost of Living Index")

In [30]:
ds_par_ms_sli.show()

+--------------------+--------------------+------------------+--------+---------+
|           Job Title|Cost of Living Index|Mean-Annual-Salary| Country|Continent|
+--------------------+--------------------+------------------+--------+---------+
|          Researcher|                18.8|30670.244000000002|Pakistan|     Asia|
|Vice President Ja...|                18.8|          102665.5|Pakistan|     Asia|
|     Project Manager|                18.8| 69043.80200000001|Pakistan|     Asia|
|                  BA|                18.8|         44364.317|Pakistan|     Asia|
|   Senior Consultant|                18.8| 77690.06199999999|Pakistan|     Asia|
| Structural Engineer|                18.8|35669.579000000005|Pakistan|     Asia|
|            Director|                18.8|129215.41600000001|Pakistan|     Asia|
|People Operations...|                18.8|47609.494000000006|Pakistan|     Asia|
|              Intern|                18.8|         20408.896|Pakistan|     Asia|
|      Senior Ma

Saving the sorted dataset with file name **Dataset with Chosen Columns Sorted by Cost of Living Index** to Google Drive

In [31]:
ds_par_ms_sli.write.csv("/content/drive/MyDrive/Cloud Technologies/Notebooks/Transformed Datasets/Dataset with Chosen Columns Sorted by Cost of Living Index", mode="overwrite", header=True)


In [32]:
ds_par_ms_sli.show()

+--------------------+--------------------+------------------+--------+---------+
|           Job Title|Cost of Living Index|Mean-Annual-Salary| Country|Continent|
+--------------------+--------------------+------------------+--------+---------+
|          Researcher|                18.8|30670.244000000002|Pakistan|     Asia|
|     Project Manager|                18.8| 69043.80200000001|Pakistan|     Asia|
|Vice President Ja...|                18.8|          102665.5|Pakistan|     Asia|
|            Director|                18.8|129215.41600000001|Pakistan|     Asia|
|   Senior Consultant|                18.8| 77690.06199999999|Pakistan|     Asia|
|     Account Manager|                18.8|44969.062000000005|Pakistan|     Asia|
|                  BA|                18.8|         44364.317|Pakistan|     Asia|
|Senior Staff Engi...|                18.8| 83978.88300000002|Pakistan|     Asia|
|              Intern|                18.8|         20408.896|Pakistan|     Asia|
|    Support Fun

Number of Distinct Jobs.

In [38]:
ds_par_ms_sli.select("Job Title").distinct().count()

4040

In [50]:
from pyspark.sql.functions import col, avg, round

Aggregated grouping of columns titled ***Mean-Annual-Salary*** and ***Cost of Living Index*** by ***Country*** and creating 2 new columns ***Average-Mean-Annual-Salary*** and ***Average-Cost-of-Living-Index*** by getting the average of the values.

In [43]:
grouped_ds = ds_par_ms_sli.groupBy("Country").agg( avg("Mean-Annual-Salary").alias("Average-Mean-Annual-Salary"),
                                                            avg("Cost of Living Index").alias("Average-Cost-of-Living-Index") )

Creating a new Dataset named ***selected_columns_ds*** to get necessary columns named ***Country***, ***Average-Cost-of-Living-Index***, ***Average-Mean-Annual-Salary***

In [44]:
selected_columns_ds = grouped_ds.select("Country","Average-Cost-of-Living-Index", "Average-Mean-Annual-Salary")

In [45]:
selected_columns_ds.show()

+-----------+----------------------------+--------------------------+
|    Country|Average-Cost-of-Living-Index|Average-Mean-Annual-Salary|
+-----------+----------------------------+--------------------------+
|   Paraguay|           25.39999999999999|         71789.41638461538|
|     Russia|                        31.0|          62786.3655708634|
|     Sweden|           59.29999999999673|         48254.74863616265|
|Philippines|                        31.0|           62378.540150743|
|   Malaysia|                        30.0|         66344.16228088799|
|  Singapore|           76.70000000000272|         69147.49893933609|
|     Turkey|           37.40000000000094|          57048.2983264177|
|       Iraq|           29.29999999999998|        55466.930485714285|
|    Germany|           62.20000000000318|        56629.849198359116|
|   Cambodia|           37.30000000000006|          56366.5553814433|
|     Jordan|          41.200000000000024|         57534.00967857143|
|     France|       

Sorting the ***slected_columns_ds*** by Cost of Living Index and making a new Dataset called ***mean_salary_by_country_sorted_by_col*** .<br>
This dataset will later be used for visualization.


In [46]:
mean_salary_by_country_sorted_by_col = selected_columns_ds.sort("Average-Cost-of-Living-Index")

In [47]:
mean_salary_by_country_sorted_by_col.show()

+----------+----------------------------+--------------------------+
|   Country|Average-Cost-of-Living-Index|Average-Mean-Annual-Salary|
+----------+----------------------------+--------------------------+
|  Pakistan|          18.799999999999596|         59269.39323615916|
|     Egypt|                        21.0|         82591.17056097578|
|     India|          21.200000000003097|         66207.91570797206|
|Bangladesh|                        22.5|         40081.14888405797|
|  Tanzania|          23.800000000000107|         53148.07243678155|
|     Syria|                        24.0|         45393.45591726622|
|  Paraguay|           25.39999999999999|         71789.41638461538|
|     Nepal|                        25.5|        28775.269160000003|
|   Ukraine|          25.900000000000965|         57452.10452765188|
|   Belarus|          26.399999999999988|              71970.035375|
|      Iran|          26.400000000000052|         61919.57839423076|
|Uzbekistan|          26.400000000

Rounding Average-Cost-of-Living-Index to 2 decimals.

In [51]:
mean_salary_by_country_sorted_by_col = mean_salary_by_country_sorted_by_col.withColumn( "Average-Cost-of-Living-Index", round(col("Average-Cost-of-Living-Index"), 2) )

Rounding Average-Mean-Annual-Salary to 2 decimals.

In [53]:
mean_salary_by_country_sorted_by_col = mean_salary_by_country_sorted_by_col.withColumn( "Average-Mean-Annual-Salary", round(col("Average-Mean-Annual-Salary"), 2) )

In [54]:
mean_salary_by_country_sorted_by_col.show()

+----------+----------------------------+--------------------------+
|   Country|Average-Cost-of-Living-Index|Average-Mean-Annual-Salary|
+----------+----------------------------+--------------------------+
|  Pakistan|                        18.8|                  59269.39|
|     Egypt|                        21.0|                  82591.17|
|     India|                        21.2|                  66207.92|
|Bangladesh|                        22.5|                  40081.15|
|  Tanzania|                        23.8|                  53148.07|
|     Syria|                        24.0|                  45393.46|
|  Paraguay|                        25.4|                  71789.42|
|     Nepal|                        25.5|                  28775.27|
|   Ukraine|                        25.9|                   57452.1|
|   Belarus|                        26.4|                  71970.04|
|      Iran|                        26.4|                  61919.58|
|Uzbekistan|                      

Number of Partitions

In [55]:
mean_salary_by_country_sorted_by_col.rdd.getNumPartitions()

1

Saving the sorted dataset with file name **Dataset with Average Mean Annual Salary and Cost of Living Index Grouped by Country** to Google Drive

In [56]:
mean_salary_by_country_sorted_by_col.write.csv("/content/drive/MyDrive/Cloud Technologies/Notebooks/Transformed Datasets/Dataset with Average Mean Annual Salary and Cost of Living Index Grouped by Country", mode="overwrite", header=True)


Exploratory Graph in the form of Scattered Plot.

In [77]:
import pandas as pd
import plotly.express as px

# Converting spark dataset to pandas dataframe
pandas_df = mean_salary_by_country_sorted_by_col.toPandas()

# Interactive Scatter Plot
fig = px.scatter(
                pandas_df,
                x="Average-Mean-Annual-Salary",
                y="Average-Cost-of-Living-Index",
                title="Do people in Countries with Higher Cost of Living Make More Money?",
                labels={ "Average-Mean-Annual-Salary": "Average Mean Annual Salary",
                          "Average-Cost-of-Living-Index": "Average Cost of Living Index"
                      },
                hover_data=["Country"]
                )
fig.update_layout(title={'text': "Do people in Countries with Higher Cost of Living Make More Money?", 'x': 0.5, 'xanchor': 'center'})
fig.show()



In [88]:
import plotly.graph_objects as go


fig = go.Figure()

for country in pandas_df['Country'].unique():
    country_df = pandas_df[pandas_df['Country'] == country]
    fig.add_trace(go.Scatter(
        x=country_df['Average-Mean-Annual-Salary'],
        y=country_df['Average-Cost-of-Living-Index'],
        mode='markers',
        marker= dict(color='green'),
        name=country,
        visible=(country == pandas_df['Country'].unique()[0]) # Initially show only the first country
    ))

# Buttons for dropdown menu
buttons = []
for i, country in enumerate(pandas_df['Country'].unique()):
    visibility = [False] * len(pandas_df['Country'].unique())
    visibility[i] = True
    button = dict(
        label=country,
        method="update",
        args=[{"visible": visibility},
              {"title": f"Scatter Plot - {country}"}]
    )
    buttons.append(button)

# Button for all countries
buttons.insert(0, dict(label="All",
                       method="update",
                       args=[{"visible": [True] * len(pandas_df['Country'].unique())},
                             {"title": "Do people in Countries with Higher Cost of Living Make More Money?"}]))


fig.update_layout(
    updatemenus=[dict(
        active=0,  # Default to showing all countries
        buttons=buttons,
        x=0.0,
        xanchor='left',
        y=1.1,
        yanchor='top'
    )],
    title={'text': "Do people in Countries with Higher Cost of Living Make More Money?", 'x': 0.5, 'xanchor': 'center'}
)

for trace in fig.data:
  trace.visible = 'legendonly'
fig.show()